In [9]:
import numpy as np
import pandas as pd

import os, sys

import gzip
import cPickle as pickle

import calendar

import matplotlib
% matplotlib inline

In [20]:
in_dir = "/home/data/kaggle-zillow/"
out_dir= in_dir + "processed"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Read in and format data for analysis

In [3]:
# read in raw data

print( "\nReading data from disk ...")
prop = pd.read_csv(in_dir + 'properties_2016.csv')
train = pd.read_csv(in_dir + "train_2016_v2.csv", \
                    parse_dates=["transactiondate"])
sampl = pd.read_csv(in_dir + "sample_submission.csv").rename(
            columns={"ParcelId":"parcelid"})


Reading data from disk ...


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.shape, prop.shape, sampl.shape

((90275, 3), (2985217, 58), (2985217, 7))

The only variation in the columns of the test data is across months. So it would make sense to make 6 separate predictions, in which all that changes in the feature matrix is the month column. 

In [6]:
sampl.head()

,parcelid,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [40]:
print "\n Reading in variable info ..."

xl = pd.ExcelFile(in_dir + 'zillow_data_dictionary.xlsx')
xl.sheet_names  # see all sheet names

vars_dict = {}

for sheet in xl.sheet_names:
    df = xl.parse(sheet)
    vars_dict[sheet.lower()] = dict(zip(df.iloc[:,0], df.iloc[:,1]))
    
# save dictionary to disc
with gzip.open(out_dir + "/vars_dict.pickle.gz", "w") as f:
    pickle.dump(vars_dict, f)


 Reading in variable info ...


In [58]:
train_df.columns.values

array(['parcelid', 'logerror', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'buildingqualitytypeid', 'calculatedbathnbr',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13',
       'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolsizesum',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt',
       'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17',
       'yardbuildingsqft26', 'yearbuilt', 'numberofstories',
       'fireplaceflag', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyflag',
       'taxdelinquencyyear', 'censustractandblock',
       'airconditioningtypeid_0', 'airconditioningtypeid_1',
  

In [59]:
train_df['regionidneighborhood'].nunique()

494

In [45]:
def format_data(df, retain_cols=None):
    train_df = df.copy()
    if retain_cols is not None:
        train_df = train_df[[c for c in train_df.columns if c in retain_cols]]

    # remove variables with too many categories
    for c in ['propertyzoningdesc', 'propertycountylandusecode']:
        if c in train_df.columns:
            train_df.drop(c, axis=1, inplace=True)

    # remove variables with no variation
    tr_std = train_df.std()
    train_df.drop(tr_std[tr_std==0].index, axis=1, inplace=True)

    # format categorical variables

    train_df['taxdelinquencyflag'] = train_df['taxdelinquencyflag'].apply(lambda x: 1 if x=='Y' else 0)
    train_df['fireplaceflag'] = train_df['fireplaceflag'].apply(lambda x: 1 if x==True else 0)
    train_df['hashottuborspa'] = train_df['hashottuborspa'].apply(lambda x: 1 if x==True else 0)

    categ_cols = ['parcelid'] + [c for c in train_df.columns 
                      if c in vars_dict.keys() or train_df[c].dtype==np.dtype('O') ]
    
    # some categorical variables are actually codes (FIPS, ZIP, Census Blocks, etc)
    # the problem is that some of these codes are very numerous!
    # perhaps just remove them?
    categ_cols += ['fips', 'rawcensustractandblock', 'regionidcounty', 'regionidneighborhood', 'regionidzip']

    print train_df[categ_cols].count()

    for c in categ_cols:
        train_df[c] = train_df[c].apply(lambda x: 0 if np.isnan(x) else x).astype(str)
        
    # format date fields
    if 'transactiondate' in train_df.columns:
        train_df['transactionmonth'] = train_df['transactiondate'].apply(\
                                                        lambda x: calendar.month_abbr[x.month])
    #     # we don't have access to transaction day in the test (submission) set
    #     train_df['transactionday'] = train_df['transactiondate'].apply(\
    #                                                     lambda x: int(x.day))
    #     train_df['transactiondayofweek'] = train_df['transactiondate'].apply(\
    #                                                     lambda x: calendar.day_abbr[x.dayofweek])
        categ_cols += ['transactionmonth']#, 'transactionday', 'transactiondayofweek']
        train_df.drop("transactiondate", axis=1, inplace=True)
    
    # fill missing values for some variables with 0
    # it makes more sense that for attributes like pools and fireplaces,
    # which may not be populated because they don't exist

    vars_fill_zero = ['poolsizesum', 'basementsqft', 'fireplacecnt', 
                      'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
                      'calculatedbathnbr', 'finishedfloor1squarefeet',
                      'finishedsquarefeet13', 'finishedsquarefeet15', 
                      'finishedsquarefeet50', 'finishedsquarefeet6', 
                      'threequarterbathnbr', 'yardbuildingsqft17',
                      'yardbuildingsqft26'
                      ]
    for c in vars_fill_zero:
        train_df[c] = train_df[c].fillna(0)

    # replace missing values for other variables with column medians
    median_values = train_df.median(axis=0)
    train_df = train_df.fillna(median_values, inplace=True)

    # years from 2016 of property being tax delinquent 
    train_df['taxdelinquencyyear'] = train_df['taxdelinquencyyear'].apply(\
                                        lambda x: 2016-(2000+x if x < 16 else 1900+x))
    
    train_df_dum = pd.get_dummies(train_df, columns=categ_cols, drop_first=False)
    return train_df_dum, train_df.columns.values.tolist()

In [46]:
# join in property and score data
train_df = pd.merge(train, prop, on='parcelid', how='left')

# format training data
train_df, final_vars = format_data(train_df)

train_df.to_csv(out_dir + "/train_formatted.csv")

airconditioningtypeid       28781
architecturalstyletypeid      261
heatingorsystemtypeid       56080
propertylandusetypeid       90275
typeconstructiontypeid        299
fips                        90275
dtype: int64


In [51]:
train_df['regionidzip'].value_counts().shape

(388,)

In [57]:
train_df['regionidcity'].value_counts()

12447.0     20559
25218.0      3389
46298.0      3000
52650.0      2132
54311.0      2128
5534.0       2035
40227.0      1949
16764.0      1800
34278.0      1383
27110.0      1266
12773.0      1222
47019.0      1187
13150.0      1120
47568.0      1097
45457.0      1052
24812.0      1037
33252.0      1034
34543.0       949
51239.0       932
54722.0       929
53571.0       923
25459.0       904
24832.0       891
32380.0       820
13693.0       815
37086.0       791
33612.0       772
20008.0       756
21412.0       729
15554.0       693
            ...  
114828.0       50
30399.0        49
113412.0       47
118875.0       40
16961.0        39
16389.0        36
272578.0       33
3491.0         30
32753.0        30
25468.0        27
38980.0        25
6822.0         24
13232.0        21
42091.0        20
114834.0       18
34037.0        16
25271.0        14
33312.0        14
25621.0        14
14906.0        13
31134.0        12
53162.0         9
32927.0         5
118880.0        4
10815.0   

In [54]:
train_df['rawcensustractandblock'].value_counts().shape

(42647,)

In [55]:
train_df['rawcensustractandblock'].head()

0    6.037107e+07
1    6.059052e+07
2    6.037464e+07
3    6.037296e+07
4    6.059042e+07
Name: rawcensustractandblock, dtype: float64

In [53]:
train_df.columns.values

array(['parcelid', 'logerror', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'buildingqualitytypeid', 'calculatedbathnbr',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13',
       'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolsizesum',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt',
       'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17',
       'yardbuildingsqft26', 'yearbuilt', 'numberofstories',
       'fireplaceflag', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyflag',
       'taxdelinquencyyear', 'censustractandblock',
       'airconditioningtypeid_0', 'airconditioningtypeid_1',
  

In [47]:
# format test data

test_df  = pd.merge(sampl, prop, on='parcelid', how='left').drop(smpl_cols, axis=1)

test_df, _ = format_data(test_df, retain_cols=final_vars)

test_df.to_csv(out_dir + "/sample_formatted.csv")

airconditioningtypeid        811519
architecturalstyletypeid       6061
heatingorsystemtypeid       1806401
propertylandusetypeid       2973780
typeconstructiontypeid         6747
fips                        2973780
dtype: int64


In [60]:
smpl_cols

{'201610': 'transactionmonth_Oct',
 '201611': 'transactionmonth_Nov',
 '201612': 'transactionmonth_Dec',
 '201710': 'transactionmonth_Oct',
 '201711': 'transactionmonth_Nov',
 '201712': 'transactionmonth_Dec'}